

 <h1>
Welcome to the Math Question Answer Verification Competition! 🚀

The goal is to fine-tune a Llama-3-8B model to predict if a given solution to a math problem is correct or not. Your model should output True if the solution is correct, and False otherwise.

This notebook is a starter guide designed to get you up and running quickly. We'll walk through a simplified training process using a small subset of the data (5,000 examples) and lightweight parameters. The main goal here is to understand the complete workflow, from loading data to generating a submission file, not to achieve a top score.

Good luck, and have fun! 🎉

## **Step 1: Install Necessary Libraries**

First, we need to install the required Python libraries. We'll be using the unsloth library, which provides highly efficient, memory-saving training methods for large language models, making it possible to fine-tune powerful models on a single free-tier GPU. We'll also install xformers for further optimization.


In [1]:
#%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" "trl<0.9.0" "peft<0.12.0" "accelerate<0.32.0" "bitsandbytes<0.44.0" "transformers<4.43.0"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-xp5hnbv_/unsloth_4980763c19774a7f9d4f2e41481c307e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-xp5hnbv_/unsloth_4980763c19774a7f9d4f2e41481c307e
  Resolved https://github.com/unslothai/unsloth.git to commit f2b304bd69ddb034fd34e5d663319c0d2fbe4ae4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.1 MB/s eta 0:00:00
  Using cached bitsandbytes-0.48.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached trl-0.23.0-py3-none-any.whl.metadata (11 kB)
Using cached bitsandbytes-0.48.1-py3-none-manylinux_2_24_x86_64.whl (60.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 125.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━

## **Step 2: Load the Model and Tokenizer**

Next, we'll load the Llama-3-8B model, which is the only model permitted for this competition. We'll use Unsloth's FastLanguageModel to handle this efficiently.

A key technique we'll use is 4-bit quantization (load_in_4bit = True). Think of this as compressing the model's knowledge into a much smaller file size. This significantly reduces the amount of GPU memory required, allowing us to fine-tune this large model even on a free platform like Google Colab.



In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1500  # Choose any sequence length
dtype = None  # This will auto-detect the best data type for your GPU
load_in_4bit = True  # Use 4-bit quantization to save memory

# Load the model and tokenizer from Hugging Face
# Note: We use the base model, not a 4-bit pre-quantized one,
# to ensure we start from the official weights.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B", # Competition-approved model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.11: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

## **Step 3: Prepare the Dataset**

This is a crucial step where we format our data into a structure the model can learn from. The process involves three parts:

1.  **Loading**: We'll load the official competition dataset from Hugging Face.
2.  **Splitting**: The full dataset is massive. For this starter notebook, we'll create a much smaller, more manageable version to speed things up: **5,000 samples for training** and **500 for validation**.
3.  **Prompting**: We will format each data sample into a clear instructional prompt. This helps the model understand its role as a mathematician verifying a solution.



In [3]:
from datasets import load_dataset

# Load the full training dataset
full_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

# Shuffle the dataset for randomness and create our smaller splits
# shuffled_dataset = full_dataset.shuffle(seed=42)
# train_dataset = shuffled_dataset.select(range(5000))      # Use the first 5,000 for training
# validation_dataset = shuffled_dataset.select(range(5000, 5500)) # Use the next 500 for validation

shuffled_dataset = full_dataset.shuffle(seed=42)
train_dataset = shuffled_dataset.select(range(50000))
validation_dataset = shuffled_dataset.select(range(50000, 55000))

# Full train and test set
# dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp")
# train_dataset = dataset['train']
# validation_dataset = dataset['test']

# print(f"Training set size: {len(train_dataset)}")
# print(f"Validation set size: {len(validation_dataset)}")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [5]:
# The instructional prompt template for training
# training_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
# Question:
# {}
# Solution:
# {}
# Output:
# {}"""

training_prompt = """You are a meticulous math expert. Your task is to verify the provided solution to a math problem.
First, provide a brief, step-by-step analysis of the solution to verify its correctness. Check for logical fallacies, calculation errors, or incorrect application of concepts.
After your analysis, conclude with the final verdict on a new line: the single word 'True' if the solution is correct, or 'False' if it is incorrect.

Question:
{}

Solution:
{}

Output:
{}"""

# We must add an End Of Sequence (EOS) token to tell the model when a completion is finished.
EOS_TOKEN = tokenizer.eos_token

# This function formats our data samples into the prompt template.
def formatting_prompts_func(examples):
    questions = examples["question"]
    solutions = examples["solution"]
    outputs = examples["is_correct"]
    texts = []
    for question, solution, output in zip(questions, solutions, outputs):
        # Format the prompt and add the EOS token
        text = training_prompt.format(question, str(solution), str(output)) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }

# Apply the formatting function to our training dataset
formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
formatted_eval_dataset = validation_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

## **Step 4: Configure LoRA and Set Up the Trainer**

### **LoRA Configuration**

Instead of training the entire model (which has billions of parameters), we'll use a technique called **Lo**w-**R**ank **A**daptation (LoRA). 🎛️

Think of it like this: rather than rewriting an entire textbook, we're just adding small, efficient "sticky notes" (the LoRA adapters) to update the model's knowledge. This is much faster and requires significantly less memory. We'll use a small **rank** (`r = 8`) to keep the training process light and quick for this starter notebook.


In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # A small rank for lighter training
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 128, # A common practice is to set alpha = 2 * r
    lora_dropout = 0.05,#0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.10.11 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.



### **SFTTrainer Setup**

Now we'll set up the `SFTTrainer` (Supervised Fine-tuning Trainer). This is the main tool from the `trl` library that will handle the entire training loop for us. We'll give it our model, tokenizer, dataset, and a set of training instructions, such as the batch size and number of epochs.

We will train for just **one epoch** (a single pass over our 5,000-sample dataset) to keep this demonstration fast.

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train_dataset,
    eval_dataset = formatted_eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 8, #2,
        gradient_accumulation_steps = 8, #4,
        #warmup_steps = 80, #5,
        warmup_ratio = 0.05,
        num_train_epochs = 3,
        learning_rate = 2e-4, #1e-5
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",#"linear",
        seed = 42,
        output_dir = "outputs",
        report_to = "none",

        eval_strategy="steps",
        eval_steps=50,
        save_strategy = "epoch",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/50000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/5000 [00:00<?, ? examples/s]

## **Step 5: Start Training\!**

Now, we'll call the `train()` function on our `trainer` object. This will kick off the fine-tuning process. Based on our settings, this will run for one full epoch over our 5,000 examples.

Grab a coffee, as this will take a few minutes\! ☕


In [8]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50,000 | Num Epochs = 3 | Total steps = 2,346
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 8 x 1) = 64
 "-____-"     Trainable parameters = 167,772,160 of 8,198,033,408 (2.05% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
50,0.648600,0.621218
100,0.597000,0.594635
150,0.580400,0.580430
200,0.557500,0.565237
250,0.555300,0.552412
300,0.539300,0.538160
350,0.553900,0.527301
400,0.532200,0.514665


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


Step,Training Loss,Validation Loss
50,0.648600,0.621218
100,0.597000,0.594635
150,0.580400,0.580430
200,0.557500,0.565237
250,0.555300,0.552412
300,0.539300,0.538160
350,0.553900,0.527301
400,0.532200,0.514665
450,0.503000,0.501060
500,0.499500,0.489905


TrainOutput(global_step=2346, training_loss=0.39372399584005147, metrics={'train_runtime': 41974.6442, 'train_samples_per_second': 3.574, 'train_steps_per_second': 0.056, 'total_flos': 3.820641362098127e+18, 'train_loss': 0.39372399584005147, 'epoch': 3.0})


## **Step 6: Inference and Evaluation**

Now that our model is trained, we need to test it on our validation set. We'll use a slightly different prompt for inference—one where we leave the `Output:` section blank for the model to complete.

Let's test it on a single example from our validation set to see what it predicts.

In [24]:
# Prepare the model for faster inference
FastLanguageModel.for_inference(model)

# Create the prompt template for inference (no answer included)
# inference_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
# Question:
# {}
# Solution:
# {}
# Output:
# """

inference_prompt = """You are a meticulous math expert. Your task is to verify the provided solution to a math problem.
First, provide a brief, step-by-step analysis of the solution to verify its correctness. Check for logical fallacies, calculation errors, or incorrect application of concepts.
After your analysis, conclude with the final verdict on a new line: the single word 'True' if the solution is correct, or 'False' if it is incorrect.

Question:
{}

Solution:
{}

Output:
"""

# Select a sample from the validation set
example = validation_dataset[10] # You can change the index (e.g., to 1, 2, 50)
question = example["question"]
solution = example["solution"]

# Format the prompt with the validation data
inputs = tokenizer(
[
    inference_prompt.format(question, str(solution))
], return_tensors = "pt").to("cuda")

# Generate the model's response
outputs = model.generate(**inputs, max_new_tokens = 8, use_cache = True)
response = tokenizer.batch_decode(outputs)

# Print the results
print("#### QUESTION ####")
print(question)
print("\n#### SOLUTION ####")
print(solution)
print("\n#### MODEL'S PREDICTION ####")
# We process the output to show only the generated text
print(response[0].split("Output:\n")[1])
print("\n#### CORRECT ANSWER ####")
print(example["is_correct"])

#### QUESTION ####
Spot's doghouse has a regular hexagonal base that measures one yard on each side. He is tethered to a vertex with a two-yard rope. What is the area, in square yards, of the region outside the doghouse that Spot can reach? Express your answer in terms of $\pi$.

#### SOLUTION ####
The area outside the doghouse that Spot can reach is two-thirds of the total area of the doghouse.
To compute the area of the doghouse we need to compute the area of a regular hexagon with side one.
<llm-code>
import sympy as sp

# let's compute the area of a regular hexagon with side 1
r = sp.sqrt(3)/2
hex_area = (r**2 * sp.pi)/2

# now we need to multiply the area by 2/3 to get the area that Spot can reach
spot_area = 2/3 * hex_area

# print the area
print(spot_area)
</llm-code>
<llm-code-output>
0.25*pi
</llm-code-output>
So the area that Spot can reach is $\boxed{\frac{1}{4} \pi}$.

#### MODEL'S PREDICTION ####
False<|end_of_text|>

#### CORRECT ANSWER ####
True


In [25]:
from tqdm import tqdm

print("Starting evaluation on the validation dataset...")
validation_dataset = shuffled_dataset.select(range(50000, 51000))

all_predictions = []
all_ground_truths = []

def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

for example in tqdm(validation_dataset):
    question = example["question"]
    solution = example["solution"]
    ground_truth = example["is_correct"]

    prompt = inference_prompt.format(question, str(solution))

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)

    response_text = tokenizer.batch_decode(outputs)[0]

    prediction = parse_output(response_text)

    all_predictions.append(prediction)
    all_ground_truths.append(ground_truth)

# --- Calculating Accuracy ---

correct_count = 0
total_count = len(all_ground_truths)

for pred, truth in zip(all_predictions, all_ground_truths):
    if pred == truth:
        correct_count += 1

accuracy = correct_count / total_count

print(f"\n--- Validation Results ---")
print(f"Total samples: {total_count}")
print(f"Correct predictions: {correct_count}")
print(f"Accuracy: {accuracy * 100:.2f}%")

Starting evaluation on the validation dataset...


100%|██████████| 1000/1000 [04:29<00:00,  3.71it/s]


--- Validation Results ---
Total samples: 1000
Correct predictions: 870
Accuracy: 87.00%


Original: \
--- Validation Results ---
Total samples: 500
Correct predictions: 321
Accuracy: 64.20%


## **Step 7: Generate Submission File**

This is the final step\! We will now run our fine-tuned model on the official `test` dataset.

We will loop through each example in the test set, generate a prediction, and format the results into a CSV file with two columns: `ID` and `is_correct`, as required by the competition.


In [26]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset

# --- Load dataset ---
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")

predictions = []
all_ground_truth = []

def parse_output(response_text):
    output_part = response_text.split("Output:\n")[-1]
    return "true" in output_part.lower()

correct_count = 0
total = len(test_dataset)

bar = tqdm(total=total, desc="Evaluating", unit="sample")

for i, example in enumerate(test_dataset):
    question = example["question"]
    solution = example["solution"]
    ground_truth = example["is_correct"]

    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    pred = parse_output(response_text)
    predictions.append(pred)
    all_ground_truth.append(ground_truth)

    if pred == ground_truth:
        correct_count += 1

    running_acc = correct_count / (i + 1)
    bar.set_postfix({"acc": f"{running_acc * 100:.2f}%"})
    bar.update(1)

bar.close()

# --- Final results ---
accuracy = correct_count / len(all_ground_truth)
print(f"\n--- Validation Results ---")
print(f"Total samples: {len(all_ground_truth)}")
print(f"Correct predictions: {correct_count}")
print(f"Accuracy: {accuracy * 100:.2f}%")

# --- Save submission ---
submission = pd.DataFrame({
    "ID": range(len(predictions)),
    "is_correct": predictions
})
submission.to_csv("submission.csv", index=False)
print("\nSubmission file 'submission.csv' created successfully!")

Evaluating: 100%|██████████| 10000/10000 [45:09<00:00,  3.69sample/s, acc=28.27%]


--- Validation Results ---
Total samples: 10000
Correct predictions: 2827
Accuracy: 28.27%

Submission file 'submission.csv' created successfully!


In [23]:
import pandas as pd
from tqdm import tqdm

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    # Format the prompt
    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

  7%|▋         | 693/10000 [03:05<41:27,  3.74it/s]


KeyboardInterrupt: 

# SAVE THE MODEL TO DRIVE AND RUN INFERENCE
Add code to save the model checkpoint to Google Drive, load the model from the checkpoint, and generate the final submission CSV file.

## Mount google drive

### Subtask:
Mount Google Drive to save the model checkpoint.


**Reasoning**:
Mount Google Drive to save the model checkpoint.



In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Save model checkpoint

### Subtask:
Save the trained model checkpoint to the specified path in Google Drive.


**Reasoning**:
Define the save path and save the model and tokenizer to Google Drive.



In [10]:
import os

# Define the path to save the model checkpoint in Google Drive
save_path = "/content/drive/MyDrive/DL-Fall-25/llama3_8b_math_verifier_checkpoint_2"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model checkpoint and tokenizer saved to: {save_path}")

Model checkpoint and tokenizer saved to: /content/drive/MyDrive/DL-Fall-25/llama3_8b_math_verifier_checkpoint_2


## Load model from checkpoint

### Subtask:
Load the model from the saved checkpoint.


**Reasoning**:
Load the model and tokenizer from the saved checkpoint path in Google Drive and prepare the model for inference.



In [20]:
# Define the path where the model checkpoint was saved in Google Drive
save_path = "/content/drive/MyDrive/DL-Fall-25/llama3_8b_math_verifier_checkpoint_2"

# Load the model and tokenizer from the saved path
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = save_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Prepare the loaded model for faster inference
FastLanguageModel.for_inference(model)

print(f"Model and tokenizer loaded from: {save_path}")

==((====))==  Unsloth 2025.10.11: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model and tokenizer loaded from: /content/drive/MyDrive/DL-Fall-25/llama3_8b_math_verifier_checkpoint_2


## Generate submission file

### Subtask:
Generate the submission CSV file using the loaded model.


**Reasoning**:
Generate the submission CSV file by iterating through the test dataset, generating predictions using the loaded model, and saving the results to a pandas DataFrame.



In [ ]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# Create the prompt template for inference (no answer included)
inference_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
Solution:
{}
Output:
"""

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    # Format the prompt
    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

100%|██████████| 10000/10000 [1:33:05<00:00,  1.79it/s]


Submission file 'submission.csv' created successfully!
You can now download this file and submit it to the Kaggle competition.


In [ ]:
# ---
#
# Delete model trainer and GPU Cache
# ---

del model
del trainer

import torch
import gc

gc.collect()
torch.cuda.empty_cache()

print("Model and Trainer deleted. GPU cache cleared.")
print(f"Current GPU Memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Current GPU Memory cached:    {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

Model and Trainer deleted. GPU cache cleared.
Current GPU Memory allocated: 12.62 GB
Current GPU Memory cached:    12.85 GB
